Web -> Bedrock -> Translate -> Comprehend -> S3

1. Use `requests` to scrap web pages and extract the text using `BeautifulSoup`.

2. Send the text to Amazon Bedrock for cleaning/summarizing.

3. Translate to English using Amazon Translate.

4. Perform sentiment analysis using Amazon Comprehend.

5. Serialize the results and upload them to S3 as json files.

Before running, please run `pip install -r requirements.txt`.


In [ ]:
import os
import json
import time
import hashlib
import logging
from typing import List, Tuple, Dict, Any

import requests
from bs4 import BeautifulSoup
import boto3
from botocore.exceptions import ClientError, BotoCoreError


In [ ]:

# logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("web-pipeline")

# Environment variable configuration (can be done using AWS Configure or Environment Variables)
AWS_REGION = os.environ.get("AWS_REGION", "eu-west-1")
S3_BUCKET = os.environ.get("RESULT_S3_BUCKET", "ceu-jiaqi-2025")
BEDROCK_MODEL_ID = os.environ.get("BEDROCK_MODEL_ID", "")  # 若不使用 Bedrock 可留空

# boto3 clients
s3 = boto3.client("s3", region_name=AWS_REGION)
translate = boto3.client("translate", region_name=AWS_REGION)
comprehend = boto3.client("comprehend", region_name=AWS_REGION)
# The Bedrock client name in boto3 is 'bedrock-runtime' (SDK support required).
bedrock = boto3.client("bedrock-runtime", region_name=AWS_REGION)


In [ ]:

# 1. Use `requests` to scrap web pages and extract the text using `BeautifulSoup`.


def fetch_page_text(url: str, timeout: int = 10) -> str:
    """
    Use requests + BeautifulSoup for simple text extraction.

    Strategy: First try the `<article>` tag; if none is found, merge all `<p>` paragraphs.

    Return to plain text (removing extra whitespace).
    """
    headers = {"User-Agent": "Mozilla/5.0 (compatible; web-pipeline/1.0)"}
    resp = requests.get(url, timeout=timeout, headers=headers)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    # Try common text containers.
    candidates = []
    article = soup.find("article")
    if article:
        candidates.append(article.get_text(separator="\n", strip=True))

    # Common class names (simplified)
    for cls in ("main", "content", "post", "article-body", "entry-content"):
        el = soup.find(class_=cls)
        if el:
            candidates.append(el.get_text(separator="\n", strip=True))

    # fallback: merge all `<p>` paragraphs
    if not candidates:
        paragraphs = soup.find_all("p")
        text = "\n\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
    else:
        # Select the longest candidate.
        text = max(candidates, key=len)

    # Clean up extra whitespace
    text = "\n".join(line.strip() for line in text.splitlines() if line.strip())
    return text


In [ ]:

# 2. Send the text to Amazon Bedrock for cleaning/summarizing.

def bedrock_clean_text(text: str, model_id: str = BEDROCK_MODEL_ID, timeout_seconds: int = 60) -> str:
    """
    If `BEDROCK_MODEL_ID` is configured, the `text` is sent to the Bedrock model, which then returns the cleaned text.
    If `model_id` is not configured, the original text is returned directly.
    """
    if not model_id:
        logger.info("No Bedrock model configured, skipping Bedrock step.")
        return text

    prompt = (
        "You are a text-cleaning assistant.\n"
        "Given a noisy HTML-extracted article text, return a cleaned, readable plaintext article.\n\n"
        "ARTICLE:\n" + text + "\n\nCLEANED ARTICLE:\n"
    )

    try:
        response = bedrock.invoke_model(
            modelId=model_id,
            contentType="text/plain; charset=utf-8",
            accept="application/json",
            body=prompt.encode("utf-8")
        )
        body_bytes = response["body"].read()
        cleaned = body_bytes.decode("utf-8")
        return cleaned
    except (ClientError, BotoCoreError) as e:
        logger.exception("Bedrock invocation failed, returning original text: %s", e)
        return text


In [ ]:
# 3. Segment long texts (because Translate/Comprehend has length limitations).

def split_text_chunks(text: str, max_chars: int = 4500) -> List[str]:
    """
    Segment by paragraph and combine into blocks not exceeding max_chars=4500.
    """
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    chunks = []
    cur = []
    cur_len = 0
    for p in paragraphs:
        if len(p) > max_chars:
            # If a single paragraph is too long, then it can be split into sentences or fixed lengths.
            for i in range(0, len(p), max_chars):
                piece = p[i:i+max_chars]
                if cur:
                    chunks.append('\n\n'.join(cur))
                    cur = []
                    cur_len = 0
                chunks.append(piece)
        else:
            if cur_len + len(p) + 2 > max_chars:
                chunks.append('\n\n'.join(cur))
                cur = [p]
                cur_len = len(p)
            else:
                cur.append(p)
                cur_len += len(p) + 2
    if cur:
        chunks.append('\n\n'.join(cur))
    return chunks


In [ ]:
# 4. Translation (send each chunk to Translate, then reassemble them)

def translate_chunks_to_english(chunks: List[str]) -> Tuple[str, str]:
    """Translate each text block into English and merge them, returning (full_translated_text, detected_source_language). 
    SourceLanguageCode is adjusted manually.
    """
    translated_parts = []
    detected_lang = None
    for chunk in chunks:
        try:
            resp = translate.translate_text(Text=chunk, SourceLanguageCode="zh",TargetLanguageCode="en")
            translated_parts.append(resp.get("TranslatedText", ""))
            if detected_lang is None:
                detected_lang = resp.get("SourceLanguageCode")
        except (ClientError, BotoCoreError) as e:
            logger.exception("Translate failed for a chunk, adding original chunk instead: %s", e)
            translated_parts.append(chunk)
    return "\n\n".join(translated_parts), (detected_lang or "unknown")


In [ ]:

# 5. Sentiment analysis (using Comprehend on translated English text)

def analyze_sentiment_for_text(text: str, language_code: str = "en") -> Dict[str, Any]:
    """If the text is too long, call `detect_sentiment` block by block, then merge the scores.
    Returns the overall sentiment label (majority or score-based synthesis) and details for each block.
    """
    chunks = split_text_chunks(text, max_chars=4500)
    results = []
    # Comprehend detect_sentiment processes a piece of text each time.
    for chunk in chunks:
        try:
            resp = comprehend.detect_sentiment(Text=chunk, LanguageCode=language_code)
            results.append(resp)
        except (ClientError, BotoCoreError) as e:
            logger.exception("Comprehend detect_sentiment failed for a chunk: %s", e)
            results.append({"Sentiment": "ERROR", "SentimentScore": {}})

    # Merging logic: Averaging of all sentiment scores
    score_sum = {"Positive": 0.0, "Negative": 0.0, "Neutral": 0.0, "Mixed": 0.0}
    valid = 0
    for r in results:
        sc = r.get("SentimentScore") or {}
        if sc:
            valid += 1
            for k in score_sum.keys():
                score_sum[k] += float(sc.get(k, 0.0))
    if valid > 0:
        avg_score = {k: v / valid for k, v in score_sum.items()}
        # The sentiment with the highest average score was selected as the overall sentiment.
        overall_sentiment = max(avg_score.items(), key=lambda x: x[1])[0]
    else:
        avg_score = {}
        overall_sentiment = "UNKNOWN"

    return {
        "overall_sentiment": overall_sentiment,
        "average_scores": avg_score,
        "per_chunk": results,
    }


In [ ]:
# 6. Upload JSON to S3

def upload_json_to_s3(obj: Dict[str, Any], key: str) -> str:
    body = json.dumps(obj, ensure_ascii=False, indent=None).encode("utf-8")
    try:
        s3.put_object(Bucket=S3_BUCKET, Key=key, Body=body, ContentType="application/json; charset=utf-8")
    except ClientError:
        logger.exception("Failed to upload result to S3")
        raise
    return f"s3://{S3_BUCKET}/{key}"


In [ ]:
# 7. String all the steps together into a single `process_url` function.

def process_url(url: str, do_bedrock_clean: bool = True) -> Dict[str, Any]:
    start = time.time()
    logger.info("Processing URL: %s", url)
    raw_text = fetch_page_text(url)
    if not raw_text or len(raw_text.strip()) == 0:
        logger.warning("No textual content extracted from %s", url)
        return {"url": url, "status": "no_content"}

    cleaned_text = raw_text
    if do_bedrock_clean and BEDROCK_MODEL_ID:
        cleaned_text = bedrock_clean_text(raw_text, model_id=BEDROCK_MODEL_ID)

    chunks = split_text_chunks(cleaned_text)
    translated_text, src_lang = translate_chunks_to_english(chunks)


    sentiment = analyze_sentiment_for_text(translated_text, language_code="en")

    result = {
        "source_url": url,
        "detected_source_language": src_lang,
        "raw_text_excerpt": raw_text[:2000],
        "cleaned_text_excerpt": cleaned_text[:2000],
        "translated_text_excerpt": translated_text[:2000],
        "sentiment": sentiment,
        "meta": {
            "char_counts": {
                "raw": len(raw_text),
                "cleaned": len(cleaned_text),
                "translated": len(translated_text),
            },
            "processing_time_sec": time.time() - start,
        }
    }

    # Use the URL's SHA1 hash + timestamp as the object key.
    key = f"results/{hashlib.sha1(url.encode()).hexdigest()}_{int(time.time())}.json"
    s3_uri = upload_json_to_s3(result, key)
    result["s3_uri"] = s3_uri
    logger.info("Finished %s -> %s", url, s3_uri)
    return result


In [ ]:
# 8. Demo: Handling single or multiple URLs

if __name__ == "__main__":
    # Put a list of URLs to be processed into urls.
    urls = [
        "https://example.com/some-article",
    ]
    for u in urls:
        try:
            out = process_url(u, do_bedrock_clean=False) 
            print(json.dumps(out, ensure_ascii=False, indent=2))
        except Exception as e:
            logger.exception("Error processing %s: %s", u, e)
